# Logistic Regression with Dask-ML
training a classifier using Dask-ML

In [12]:
import mlrun

mlrun.mlconf.dbpath = 'http://mlrun-api:8080'

## parameters


In [13]:
DESCRIPTION        = 'train a XGBoost classifier on a Dask cluster'       

IMAGE              = 'yjbds/mlrun-dask:dev'                      # custom image
JOB_KIND           = 'job'                                           

TASK_NAME          = 'user-task-train-logreg-dask'               # whatever

ARTIFACTS_PATH     = '/User/repos/demos/dask/artifacts'

DASK_CLIENT        = 'scheduler.json'
TRAIN_SET          = ('xtrain', 'ytrain') # published names in the cluster 
VALID_SET          = ('xvalid', 'yvalid')

## load and configure function

In [26]:
FUNC_PY = '/User/repos/demos/dask/code/clf_logreg_dask.py'
FUNC_YAML = '/User/repos/demos/dask/yaml/clf_logreg_dask.yaml'

HANDLER = 'clf_logreg_dask'

In [27]:
clf_xgb = mlrun.new_function(command=FUNC_PY, 
                               image=IMAGE,
                               kind=JOB_KIND)
clf_xgb.spec.build.image = IMAGE

clf_xgb.export(FUNC_YAML)
# clf_lgbm = mlrun.import_function(FUNC_YAML)

clf_xgb.apply(mlrun.mount_v3io())

clf_xgb.deploy(skip_deployed=True, with_mlrun=False)

# create and run the task
train_clf_xgb_task = mlrun.NewTask(
    TASK_NAME, 
    handler=HANDLER,  
    params={
        'dask_client': DASK_CLIENT,
        'train_set'  : TRAIN_SET,
        'valid_set'  : VALID_SET,
        'target_path': ARTIFACTS_PATH,
        'name'       : 'glm-dask-model.pkl',
        'key'        : 'glm-dask-model'})

# run
rn = clf_xgb.run(train_clf_xgb_task)

[mlrun] 2020-02-13 15:49:59,165 function spec saved to path: /User/repos/demos/dask/yaml/clf_logreg_dask.yaml
[mlrun] 2020-02-13 15:49:59,173 starting run user-task-train-logreg-dask uid=c10fcb8d3ed64488ae7aa8f447327b85  -> http://mlrun-api:8080
[mlrun] 2020-02-13 15:49:59,237 Job is running in the background, pod: user-task-train-logreg-dask-ww5h6
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1.0, max_iter=100, multi_class='ovr',
                   n_jobs=1, penalty='l2', random_state=None, solver='admm',
                   solver_kwargs=None, tol=0.0001, verbose=0, warm_start=False)
FAILED name 'coef_' is not defined

[mlrun] 2020-02-13 15:54:43,119 run executed, status=completed
Intel(R) Data Analytics Acceleration Library (Intel(R) DAAL) solvers for sklearn enabled: https://intelpython.github.io/daal4py/sklearn.html
/opt/conda/lib/python3.7/site-packages/dask/dataframe/multi.py:1059: UserWarning: Concatenating data

uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...327b85,0,Feb 13 15:50:10,completed,clf_logreg_dask,host=user-task-train-logreg-dask-ww5h6kind=jobowner=admin,,"dask_client=scheduler.jsonkey=glm-dask-modelname=glm-dask-model.pkltarget_path=/User/repos/demos/dask/artifactstrain_set=['xtrain', 'ytrain']valid_set=['xvalid', 'yvalid']",,


to track results use .show() or .logs() or in CLI: 
!mlrun get run c10fcb8d3ed64488ae7aa8f447327b85  , !mlrun logs c10fcb8d3ed64488ae7aa8f447327b85 
[mlrun] 2020-02-13 15:54:50,065 run executed, status=completed


distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/conda/lib/python3.6/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File "/conda/lib/python3.6/site-packages/distributed/client.py", line 1306, in _close
    await asyncio.wait_for(asyncio.gather(*coroutines), 2)
  File "/conda/lib/python3.6/asyncio/tasks.py", line 351, in wait_for
    yield from waiter
  File "/conda/lib/python3.6/asyncio/futures.py", line 327, in __iter__
    yield self  # This tells Task to wait for completion.
  File "/conda/lib/python3.6/asyncio/tasks.py", line 250, in _wakeup
    future.result()
  File "/conda/lib/python3.6/asyncio/futures.py", line 238, in result
    raise CancelledError
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/conda/lib/python3.6/site-packages/distributed/utils.py", line 662, in lo

In [17]:
import dask
import dask.dataframe as dd
from dask.distributed import Client
from dask_ml.linear_model import LogisticRegression

In [18]:
client = Client(scheduler_file='/User/repos/demos/dask/artifacts/scheduler.json')

/conda/lib/python3.6/site-packages/distributed/client.py:1071: VersionMismatchWarning: Mismatched versions found

lz4
+--------------------------+---------+
|                          | version |
+--------------------------+---------+
| client                   | 3.0.2   |
| scheduler                | 2.2.1   |
| tcp://10.233.64.53:45980 | 2.2.1   |
| tcp://10.233.64.54:44009 | 2.2.1   |
| tcp://10.233.64.55:41054 | 2.2.1   |
| tcp://10.233.64.56:35715 | 2.2.1   |
| tcp://10.233.64.57:40157 | 2.2.1   |
| tcp://10.233.64.58:34299 | 2.2.1   |
| tcp://10.233.64.59:42246 | 2.2.1   |
| tcp://10.233.64.60:39070 | 2.2.1   |
+--------------------------+---------+

msgpack
+--------------------------+---------+
|                          | version |
+--------------------------+---------+
| client                   | 0.6.1   |
| scheduler                | 0.6.2   |
| tcp://10.233.64.53:45980 | 0.6.2   |
| tcp://10.233.64.54:44009 | 0.6.2   |
| tcp://10.233.64.55:41054 | 0.6.2   |
| tcp://10.233.

In [19]:
xtrain = client.get_dataset('xtrain')

In [21]:
xtrain = xtrain.compute()

In [23]:
xtrain.values

array([[2.006e+03, 1.100e+01, 7.000e+00, ..., 0.000e+00, 0.000e+00,
        6.420e+02],
       [2.006e+03, 1.100e+01, 2.500e+01, ..., 1.000e+00, 1.000e+00,
        6.700e+01],
       [2.006e+03, 1.100e+01, 1.100e+01, ..., 0.000e+00, 0.000e+00,
        6.420e+02],
       ...,
       [1.998e+03, 1.200e+01, 3.100e+01, ..., 3.000e+01, 3.500e+01,
        6.920e+02],
       [1.998e+03, 1.200e+01, 9.000e+00, ..., 1.600e+01, 3.500e+01,
        3.320e+02],
       [1.998e+03, 1.200e+01, 6.000e+00, ..., 6.600e+01, 3.500e+01,
        1.510e+02]])